# Example: Reading vector data

This example illustrates the how to read raster data using the HydroMT [DataCatalog](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.html) with the `vector` or `vector_table`  drivers.

In [ ]:
import pandas as pd

import hydromt

# Download artifacts for the Piave basin to `~/.hydromt_data/`.
data_catalog = hydromt.DataCatalog(data_libs=["artifact_data=v1.0.0"])

## Pyogrio driver

To read vector data and parse it into a [geopandas.GeoDataFrame](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html) object we use the [geopandas.read_file](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html) method, see the [geopandas documentation](https://geopandas.org/en/stable/docs/user_guide/io.html#reading-spatial-data) for details. Geopandas supports many file formats, see below. For large datasets we recommend using data formats which contain a spatial index, such as 'GeoPackage (GPKG)' or 'FlatGeoBuf' to speed up reading spatial subsets of the data. Here we use a spatial subset of the [Database of Global Administrative Areas (GADM)](https://gadm.org/download_world.html) level 3 units.

In [ ]:
# inspect data source entry in data catalog yaml file
data_catalog.get_source("gadm_level3")

We can load any GeoDataFrame using the [get_geodataframe()](https://deltares.github.io/hydromt/latest/_generated/hydromt.data_catalog.DataCatalog.get_geodataframe.html) method of the DataCatalog. Note that if we don't provide any arguments it returns the full dataset with nine data variables and for the full spatial domain. Only the data coordinates are actually read, the data variables are still loaded lazy.

In [ ]:
gdf = data_catalog.get_geodataframe("gadm_level3")
print(f"number of rows: {gdf.index.size}")
gdf.head()

We can request a (spatial) subset data by providing additional `variables` and `bbox` / `geom` arguments. Note that this returns less polygons (rows) and only two columns with attribute data,

In [ ]:
gdf_subset = data_catalog.get_geodataframe(
    "gadm_level3", bbox=gdf[:5].total_bounds, variables=["GID_0", "NAME_3"]
)
print(f"number of rows: {gdf_subset.index.size}")
gdf_subset.head()

## Vector_table driver

In [ ]:
# create example point CSV data with funny `x` coordinate name and additional column
import numpy as np

path = "tmpdir/xy.csv"
df = pd.DataFrame(
    columns=["x_centroid", "y"],
    data=np.vstack([gdf_subset.centroid.x, gdf_subset.centroid.y]).T,
)
df["name"] = gdf_subset["NAME_3"]
df.to_csv(path)  # write to file
df.head()

In [ ]:
# Because the data we wrote does not live in the root of the data catalog we'll have to
# start with a new one
data_catalog = hydromt.DataCatalog(data_libs=None)

# Create data source entry for the data catalog for the new csv data
# NOTE that we add specify the name of the x coordinate with the `x_dim` argument, while
# the y coordinate is understood by HydroMT.
data_source = {
    "GADM_level3_centroids": {
        "uri": path,
        "data_type": "GeoDataFrame",
        "driver": {"name": "geodataframe_table", "options": {"x_dim": "x_centroid"}},
        "metadata": {"crs": 4326},
    }
}
data_catalog.from_dict(data_source)
data_catalog.get_source("GADM_level3_centroids")

In [ ]:
data_catalog.get_source("GADM_level3_centroids")

In [ ]:
# we can then read the data back as a GeoDataFrame
gdf_centroid = data_catalog.get_geodataframe("GADM_level3_centroids")
print(f"CRS: {gdf_centroid.crs}")
gdf_centroid.head()

## Visualize vector data

The data can be visualized with the [.plot()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html) geopandas method. In an interactive environment you can also try the [.explore()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html) method

In [ ]:
ax = gdf.plot()
gdf_subset.plot(ax=ax, color="red")
gdf_centroid.plot(ax=ax, color="k")